In [96]:
import folium # library for maping
import json
import pandas as pd
import os
from branca.colormap import LinearColormap
from bokeh.io import export_png

In [97]:
# Resources
# Boundaries of the zipcodes
phx_geo = "resources/phoenix_zip_codes_geo_from_us_az_simplified_ms.json"
# file containing the values to plot
pv_file = os.path.join("resources","clean_income_file.csv")
# file to which we want the map to be saved
map_file = os.path.join("plots","population_density") #we will add the .html and .png extension
colorbar_caption = 'Number of tax payers'
value_clabel = 'Number of returns'
zipcode_clabel = 'Zip Code'

In [98]:
# Loading the Property values
df = pd.read_csv(pv_file)
df.head()


,Zip Code,City,Number of returns,Amount,Average Income
0,85003,Phoenix,"3,900","290,006",74360.51
1,85004,Phoenix,"2,960","232,765",78636.82
2,85006,Phoenix,"9,680","352,359",36400.72
3,85007,Phoenix,"5,300","278,695",52583.96
4,85008,Phoenix,"23,140","797,391",34459.42


In [99]:
# df = df.loc[:, ["ZipCode", "Zhvi"]]
df[zipcode_clabel] = df[zipcode_clabel].astype(str)
# lets try to sort the values
df = df.dropna() # let's just try to remove the NaN rows
df['Number of returns'].replace(regex=True,inplace=True,to_replace=r',',value=r'')
df["Number of returns"] = df["Number of returns"].astype(int)
df.count()

Zip Code             148
City                 148
Number of returns    148
Amount               148
Average Income       148
dtype: int64

In [100]:
# Transforming the data into a dictionnary (to be used by folium.GeoJson)
# One Entry by zipcode and the correspondig Zhvi value
map_dict = df.set_index(zipcode_clabel)[value_clabel].to_dict()
map_dict

{'85003': 3900,
 '85004': 2960,
 '85006': 9680,
 '85007': 5300,
 '85008': 23140,
 '85009': 15940,
 '85012': 3800,
 '85013': 9800,
 '85014': 12160,
 '85015': 15590,
 '85016': 17560,
 '85017': 14110,
 '85018': 17200,
 '85019': 10240,
 '85020': 15970,
 '85021': 16180,
 '85022': 23110,
 '85023': 15340,
 '85024': 12050,
 '85027': 17710,
 '85028': 10090,
 '85029': 19460,
 '85031': 11560,
 '85032': 30860,
 '85033': 21280,
 '85034': 2090,
 '85035': 19510,
 '85037': 19830,
 '85040': 11580,
 '85041': 24170,
 '85042': 18060,
 '85043': 12720,
 '85044': 19910,
 '85045': 3330,
 '85048': 16170,
 '85050': 13920,
 '85051': 18110,
 '85053': 13590,
 '85054': 3640,
 '85083': 7980,
 '85085': 10650,
 '85086': 17160,
 '85087': 3880,
 '85118': 5300,
 '85119': 8160,
 '85120': 10810,
 '85122': 19980,
 '85128': 5380,
 '85131': 3980,
 '85132': 7640,
 '85137': 780,
 '85138': 15070,
 '85139': 7100,
 '85140': 14330,
 '85142': 25800,
 '85143': 14280,
 '85145': 900,
 '85172': 740,
 '85173': 960,
 '85193': 1210,
 '8519

In [101]:
color_scale = LinearColormap(['yellow','red'], vmin = min(map_dict.values()), vmax = max(map_dict.values()))

In [102]:
# defining the function that will look into the color_scale to pick the color corresponding to the value
def get_color(feature):
    value = map_dict.get(feature['properties']['name'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_scale(value)

In [103]:
# Creating the map
m = folium.Map(location = [33.4484, -112.0740], zoom_start=8.5)
# Adding the colored zipcodes
folium.GeoJson(
    data = phx_geo,
    style_function = lambda feature: {
        'fillColor': get_color(feature),
        'fillOpacity': 0.7,
        'color' : 'black',
        'weight' : 1,
    }    
).add_to(m)
# Adding the colorbar
color_scale.caption = colorbar_caption
m.add_child(color_scale)

In [104]:
# Saving the map into an html file
m.save(outfile = map_file+".html")
# import subprocess
# outdir = "plots" # this directory has to exist..
# m.save("tmp.html")
# # url = "file://{}/tmp.html".format(os.getcwd())
# url = "file:///C:/Users/MariO/UA_data/PROJECT_1/tmp.html"
# print(url)
# outfn = os.path.join(outdir,"outfig.png")
# subprocess.check_call(["cutycapt","--url={}".format(url), "--out={}".format(outfn)])